In [1]:
from mss import mss
import pyautogui
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import time
from gymnasium import Env
from gymnasium.spaces import Box, Discrete
import pandas as pd
from PIL import Image
import pyautogui as inp
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers.legacy import Adam

In [2]:
class WebGame(Env):
    def __init__(self):
        self.restart_button_npy = np.load('restart_button.npy')
        super().__init__()
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(6,), dtype=np.uint8)
        self.game_location = {'top': 275, 'left': 501, 'width': 343, 'height': 103}
        self.retry_button = {'top':330, 'left':705, 'width':30, 'height':20}
        self.new_dimensions = (60, 20)
        self.cap = mss()


    def step(self, action):
        action_map = {
            0:'space',
            1:'down',
            2:'no_op'
        }
        if action != 2:
            inp.press(action_map[action])
        
        if_g_over = self.game_over()
        new_obs = self.state()
        reward = 10
        if action==0: reward-=1
        if action==1: reward-=1
        
        return new_obs, reward, if_g_over, {}

    def render(self):
        pass

    def reset(self):
        inp.moveTo(290, 290)
        inp.click()
        inp.press('enter')
        return self.state()
    
    def state(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3].astype(np.uint8)
        raw = np.mean(raw, axis=-1).astype(np.uint8)
        _, raw = cv.threshold(raw, 200, 255, cv.THRESH_BINARY)
        raw2 = raw.copy()
 
        _, thresh = cv.threshold(raw2, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
        contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        min_contour_area = 250
        filtered_contours = [cnt for cnt in contours if cv.contourArea(cnt) > min_contour_area]
        border = 8
        for contour in filtered_contours:
            x, y, w, h = cv.boundingRect(contour)
            cv.rectangle(raw2, (x- border, y - border), (x + w + border, y + h + border), (0, 0, 0), -1)

        raw2 = cv.resize(raw2, self.new_dimensions, interpolation=cv.INTER_LANCZOS4)
        _, raw2 = cv.threshold(raw2, 195, 255, cv.THRESH_BINARY)

        contours, hierarchy = cv.findContours(cv.bitwise_not(raw2), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        min_contour_area = 1
        filtered_contours = [cnt for cnt in contours if cv.contourArea(cnt) > min_contour_area]
        filtered_contours = filtered_contours[::-1]

        if (len(filtered_contours) < 1): 
            channel = np.array([100,100,100,100,100,100])
    
        elif (len(filtered_contours) == 1):
            x, y, w, h = cv.boundingRect(filtered_contours[0])
            channel = np.array([x,h,w,100,100,100])
        else:
            x1, y1, w1, h1 = cv.boundingRect(filtered_contours[0])
            x2, y2, w2, h2 = cv.boundingRect(filtered_contours[1])
            channel  = np.array([x1,h1,w1,x2,h2,w2])
            
        return channel
    
    
    def game_over(self):
        retry_img = np.array(self.cap.grab(self.retry_button))[:,:,:3]
        if np.array_equal(retry_img, self.restart_button_npy): return True
        else: return False

In [3]:
env = WebGame()


In [4]:
env.step(1)

(array([  0,  20,  60, 100, 100, 100]), 9, False, {})

In [5]:
np.set_printoptions(threshold=np.inf)
print(env.state())

[  0  20  60 100 100 100]


In [6]:
states = env.observation_space.shape
actions = env.action_space.n

In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, 6)))
    model.add(Dense(64, activation='relu', input_shape=states))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [14]:
del model

In [15]:
model = build_model(states, actions)

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 6)                 0         
                                                                 
 dense_4 (Dense)             (None, 64)                448       
                                                                 
 dense_5 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 32)                4128      
                                                                 
 dense_7 (Dense)             (None, 3)                 99        
                                                                 
Total params: 12995 (50.76 KB)
Trainable params: 12995

In [17]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [18]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=500000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [19]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000000, visualize=False, verbose=1)

2023-10-16 12:56:07.980506: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-10-16 12:56:08.037031: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_5_1/kernel/Assign' id:257 op device:{requested: '', assigned: ''} def:{{{node dense_5_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5_1/kernel, dense_5_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 5000000 steps ...
Interval 1 (0 steps performed)
    6/10000 [..............................] - ETA: 2:05 - reward: 10.0000 

/Users/utkarsh/miniconda3/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-10-16 12:56:08.645430: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_7/BiasAdd' id:217 op device:{requested: '', assigned: ''} def:{{{node dense_7/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_7/MatMul, dense_7/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-10-16 12:56:08.668487: W tensorflow/c/c_api.cc:304] Operation '{name:'count/Assign' id:490 op device:{requested: '', assigned: ''} def:{{{node count/Assign}} = AssignVariableOp[_has_manual_control_

   11/10000 [..............................] - ETA: 2:01 - reward: 10.0000

/Users/utkarsh/miniconda3/lib/python3.11/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
2023-10-16 12:56:08.863359: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_7_1/BiasAdd' id:324 op device:{requested: '', assigned: ''} def:{{{node dense_7_1/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_7_1/MatMul, dense_7_1/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


AttributeError: 'str' object has no attribute 'get_updates'

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)